In [1]:
print("Linear Algebra")

Linear Algebra


In [2]:
fruits = ["mango","orange","apple"]
print(fruits[1])

orange


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

print("All packages are installed correctly!")

import pandas as pd

All packages are installed correctly!


In [3]:
df = pd.read_csv(r"C:\Users\micha\Downloads\NBA_2024_per_game(03-01-2024).csv")

In [4]:
df['FGMi'] = df['FGA'] - df['FG']

In [5]:
df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,FGMi
0,Precious Achiuwa,C-PF,24,TOT,26,0,17.2,3.0,6.6,0.453,...,2.0,3.4,5.3,1.7,0.6,0.5,1.1,1.6,7.4,3.6
1,Precious Achiuwa,C,24,TOR,25,0,17.5,3.1,6.8,0.459,...,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7,3.7
2,Precious Achiuwa,PF,24,NYK,1,0,9.0,0.0,2.0,0.000,...,1.0,2.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
3,Bam Adebayo,C,26,MIA,23,23,34.1,8.0,15.7,0.506,...,2.1,8.3,10.4,4.0,1.1,1.0,2.7,2.4,22.0,7.7
4,Ochai Agbaji,SG,23,UTA,34,10,21.1,2.6,5.7,0.451,...,0.8,1.7,2.5,1.0,0.6,0.5,0.7,1.5,6.6,3.1


In [7]:

team_stats = df.groupby('Tm').agg({
    'FGA': 'sum',
    'ORB': 'sum',
    'TOV' : 'sum',
    'FTA': 'sum'
    
}).reset_index()

team_stats['possessions'] = (team_stats['FGA']+team_stats['ORB']-team_stats['TOV']+(.4*team_stats['FTA']))

team_stats = team_stats.rename(columns={'possessions': 'team_possessions'})

df = pd.merge(df, team_stats[['Tm', 'team_possessions']], on='Tm')



In [1]:
from sklearn.linear_model import LinearRegression

X = df[['ORB','DRB','STL','BLK','TOV','PF','FGMi','FTA']]
Y = df['team_possessions']

model = LinearRegression()
model.fit(X,Y)

print('Regression Coefficients:',model.coef_)
print(model.intercept_)
              

NameError: name 'df' is not defined

In [8]:
coefficients=model.coef_

adjusted_coefficients = coefficients.copy()
adjusted_coefficients[1] = abs(adjusted_coefficients[1])
adjusted_coefficients[2] = abs(adjusted_coefficients[2])
adjusted_coefficients[3] = abs(adjusted_coefficients[3])
adjusted_coefficients[7] = abs(adjusted_coefficients[7])
adjusted_coefficients[6] = -abs(.5*adjusted_coefficients[6])
adjusted_coefficients[4] = -abs(adjusted_coefficients[4])

df['WPC'] = sum(df[col]*coef for col,coef in zip(X.columns, adjusted_coefficients))
df['WPC'] += model.intercept_

NameError: name 'model' is not defined

In [60]:
print(df['WPC'])

0      134.398857
1      124.561950
2      123.212425
3      129.965898
4      131.863946
          ...    
533    125.212130
534    129.321642
535    131.985425
536    118.706876
537    127.995834
Name: WPC, Length: 538, dtype: float64


In [61]:
average_WPC = df['WPC'].mean()

print(f"Average WPC: {average_WPC}")

Average WPC: 127.3419178764072


In [62]:
df['WPC_adjusted'] = df['WPC'] * (100 / average_WPC)
WPC_new = df['WPC_adjusted'].mean()
print(f'WPC+:{WPC_new}')


WPC+:100.00000000000014


In [63]:
print(df[['Player', 'WPC', 'WPC_adjusted']].head(10))

              Player         WPC  WPC_adjusted
0   Precious Achiuwa  134.398857    105.541725
1         OG Anunoby  124.561950     97.816927
2         RJ Barrett  123.212425     96.757162
3      Nicolas Batum  129.965898    102.060579
4   Robert Covington  131.863946    103.551091
5          KJ Martin  125.443261     98.509008
6     Filip Petrušev  124.221022     97.549200
7  Immanuel Quickley  121.336732     95.284203
8       Daniel Theis  130.896983    102.791748
9        P.J. Tucker  129.981074    102.072496


In [64]:
top_10_WPC = df[['Player','WPC_adjusted']].sort_values(by='WPC_adjusted',ascending=False).head(10)
print(top_10_WPC)


                    Player  WPC_adjusted
44       Mitchell Robinson    124.966775
108            Rudy Gobert    122.106334
387           Clint Capela    121.645140
274          Mark Williams    119.545676
506          Anthony Davis    118.822712
168  Giannis Antetokounmpo    118.556683
341       Domantas Sabonis    116.897974
279            Joel Embiid    116.765245
245            Jalen Duren    116.735205
435            Nic Claxton    116.656278


In [81]:
df.groupby('Pos')['WPC_adjusted'].mean().sort_values(ascending=False)
df['WPC_pos_adj'] = df.groupby('Pos')['WPC_adjusted'].transform(lambda x: x * (100 / x.mean()))


In [82]:
top_10_WPC = df[['Player', 'Pos', 'WPC_pos_adj']].sort_values(by='WPC_pos_adj', ascending=False).head(10)
print(top_10_WPC)


                    Player Pos  WPC_pos_adj
168  Giannis Antetokounmpo  PF   117.591264
44       Mitchell Robinson   C   117.447276
445            Ben Simmons  PG   115.585661
108            Rudy Gobert   C   114.758953
387           Clint Capela   C   114.325510
13              RJ Barrett  SG   113.504051
274          Mark Williams   C   112.352375
142            Evan Mobley  PF   111.841555
506          Anthony Davis   C   111.672914
12          Scottie Barnes  SG   111.645500


In [83]:
top_25_WPC = df[['Player','Pos','WPC_pos_adj']].sort_values(by='WPC_pos_adj',ascending=False).head(25)
print(top_25_WPC)

                    Player Pos  WPC_pos_adj
168  Giannis Antetokounmpo  PF   117.591264
44       Mitchell Robinson   C   117.447276
445            Ben Simmons  PG   115.585661
108            Rudy Gobert   C   114.758953
387           Clint Capela   C   114.325510
13              RJ Barrett  SG   113.504051
274          Mark Williams   C   112.352375
142            Evan Mobley  PF   111.841555
506          Anthony Davis   C   111.672914
12          Scottie Barnes  SG   111.645500
19         Markquis Nowell  SG   110.356082
341       Domantas Sabonis   C   109.863990
279            Joel Embiid   C   109.739248
245            Jalen Duren   C   109.711015
435            Nic Claxton   C   109.636838
419           Aaron Gordon  PF   109.104481
134          Jarrett Allen   C   108.891686
393          Jalen Johnson  SF   108.513514
254         Ausar Thompson  SF   108.496980
365            Ivica Zubac   C   108.280318
324      Victor Wembanyama  PF   108.179081
407         Andre Drummond   C  

In [68]:
top_75_WPC = df[['Player','Pos','WPC_pos_adj']].sort_values(by='WPC_pos_adj',ascending=False).head(75)
print(top_75_WPC)

                    Player Pos  WPC_pos_adj
168  Giannis Antetokounmpo  PF   117.591264
44       Mitchell Robinson   C   117.447276
445            Ben Simmons  PG   115.585661
108            Rudy Gobert   C   114.758953
387           Clint Capela   C   114.325510
..                     ...  ..          ...
530         Gary Payton II  SG   103.493290
226         Toumani Camara  PF   103.437882
380          Aaron Wiggins  SG   103.434339
118          Grayson Allen  SG   103.424668
158          Naji Marshall  SF   103.391907

[75 rows x 3 columns]


In [69]:
wpc_pos_adj_std = df['WPC_pos_adj'].std()
print(f"Standard Deviation of WPC (adjusted): {wpc_pos_adj_std}")


Standard Deviation of WPC (adjusted): 3.8568414826199495


In [70]:
if 'Lebron James' in df['Player'].values:
    Lebron_wpc_adj = df[df['Player'] == 'Lebron James']['WPC_pos_adj'].iloc[0]
    print(f"Lebron James's Adjusted WPC: {Lebron_wpc_adj}")
else:
    print("Player not found.")

Player not found.


In [71]:
if 'Stephen Curry' in df['Player'].values:
    Steph_wpc_adj = df[df['Player'] == 'Stephen Curry']['WPC_pos_adj'].iloc[0]
    print(f"Stephen Curry's Adjusted WPC: {Steph_wpc_adj}")
else:
    print("Player not found.")

Stephen Curry's Adjusted WPC: 96.93232095215674


In [84]:
percentile_25 = df['WPC_pos_adj'].quantile(0.25)
percentile_50 = df['WPC_pos_adj'].quantile(0.50)  # This is the median
percentile_75 = df['WPC_pos_adj'].quantile(0.75)

print(f"25th Percentile: {percentile_25}")
print(f"50th Percentile (Median): {percentile_50}")
print(f"75th Percentile: {percentile_75}")


25th Percentile: 97.74636702349206
50th Percentile (Median): 99.57983185639227
75th Percentile: 101.4980078296154


In [85]:
percentile_95 = df['WPC_pos_adj'].quantile(0.95)
print(f"95th Percentile:{percentile_95}")


95th Percentile:106.88688750150546


In [86]:
df['WPC_per_100'] = (df['WPC_pos_adj'] / df['team_possessions']) * 100


In [87]:
if 'Jaylen Brown' in df['Player'].values:
    JayB_WPC = df[df['Player'] == 'Jaylen Brown']['WPC_pos_adj'].iloc[0]
    print(f"Jaylen Brown's WPC: {JayB_WPC}")
else: print("Player not Found")

Jaylen Brown's WPC: 97.43265117559139


In [88]:
if 'Jayson Tatum' in df['Player'].values:
    JayT_WPC = df[df['Player'] == 'Jayson Tatum']['WPC_pos_adj'].iloc[0]
    print(f"Jayson Tatum's WPC: {JayT_WPC}")
else: print("Player not Found")

Jayson Tatum's WPC: 100.36748959241962


In [89]:
if 'Derrick White' in df['Player'].values:
    DWhite_WPC = df[df['Player'] == 'Derrick White']['WPC_pos_adj'].iloc[0]
    print(f"Derrick White's WPC: {DWhite_WPC}")
else: print("Player not Found")

Derrick White's WPC: 102.64297838881852
